In [1]:
from moralis import evm_api
import requests
from web3 import Web3
import time
import pandas as pd
import web3
import random
import json

In [2]:
# 填入相应的 Key
api_key = "..."
CHAINLIST  = ['eth']
timestamp_list = [1685343054,]

def acquire_block_height():
    for chain in CHAINLIST:
        for timestamp in timestamp_list:
            #time.sleep(0.5)
            
            params = {
                "chain": chain,
                "date": str(timestamp),
            }
            
            result = evm_api.block.get_date_to_block(
                api_key=api_key,
                params=params,
            )
            
            return result

block_height = acquire_block_height()

In [5]:
block_height

{'block': 17362974,
 'date': '2023-05-29T06:50:54+00:00',
 'timestamp': 1685343054,
 'block_timestamp': '2023-05-29T06:50:47.000Z',
 'hash': '0xebef005037a453c5db3788a6a41032d2b441e117c63b6557c1535b7ac1a52122',
 'parent_hash': '0x23d0be634fcaf22d78c3c9d61d0f906de37d92b3d1ceeec278ca42340dca95e4'}

In [6]:
Web3.to_hex(b'oneInchV5FeeDynamic')
Web3.to_text(0x6f6e65496e6368563546656544796e616d6963)

'oneInchV5FeeDynamic'

In [7]:
url = "https://api.chainbase.online/v1/dw/query"

payload = {"query":"SELECT\r\n    *\r\nfrom\r\n    ethereum.transactions\r\nWHERE\r\n    to_address = \"0x881d40237659c251811cec9c364ef91dc08d300c\" and block_number < 17342324\r\nORDER BY\r\n    block_number DESC\r\nLIMIT\r\n    1000"}

headers = {
    "x-api-key": "..."
}

response = requests.post(url, json=payload, headers=headers)

#print(response.text)


In [8]:
metamask_tx = response.json()['data']['result']

In [9]:
#metamask_tx = pd.Dataframe(colume=['Protocols', 'Count'])

agg_list = {}

for tx in metamask_tx:
    agg_name = Web3.to_text('0x'+tx['input'][330:368])
    if agg_name not in agg_list:
        agg_list[agg_name] = 0
    else:
        #tmp = 
        agg_list[agg_name] += 1


In [10]:
agg_list

{'oneInchV5FeeDynamic': 748,
 'airswapLight3FeeDyn': 85,
 '0xFeeDynamic\x00\x00\x00\x00\x00\x00\x00': 65,
 'pmmFeeDynamicv4\x00\x00\x00\x00': 72,
 'openOceanFeeDynamic': 25}

In [11]:
#1inch交易分析
url = "https://api.chainbase.online/v1/dw/query"

payload = {"query":"SELECT\r\n    *\r\nfrom\r\n    ethereum.transactions\r\nWHERE\r\n    to_address = '0x881d40237659c251811cec9c364ef91dc08d300c' AND input Like '%6f6e65496e6368563546656544796e616d6963%'\r\nORDER BY\r\n    block_number DESC\r\nLIMIT\r\n    1000"}

headers = {
    "x-api-key": "..."
}

response = requests.post(url, json=payload, headers=headers)
one_inch_tx = response.json()['data']['result']

In [12]:
# 使用方法统计
oneinch_swap = '12aa3caf'
oneinch_unoswap = '0502b1c5'
oneinch_unoswap_to = 'f78dc253'
oneinch_univ3 = 'e449022e'
oneinch_univ3_to = 'bc80f1a8'
oneinch_clipper = '84bd6d29'
oneinch_fillorder = '70ccbd31'

func_list = [oneinch_swap, oneinch_unoswap, oneinch_unoswap_to, oneinch_univ3, oneinch_univ3_to, oneinch_clipper, oneinch_fillorder]
func_distribute = {}


#func_list[1] in one_inch_tx[0]['input']

for tx in one_inch_tx:
    for func in func_list:
        if func in tx['input'] and func not in func_distribute:
            func_distribute[func] = 0
        elif func in tx['input'] and func in func_distribute:
            func_distribute[func] += 1

In [13]:
func_distribute

# 结论是小狐狸用户主要使用用到 CallUniswap & CallUniswapv3 方法

{'e449022e': 220, '0502b1c5': 688, '12aa3caf': 82, '84bd6d29': 0}

In [420]:
def one_inch_unoswap_resolve(tx):
    if tx['input'][98:138] == '0000000000000000000000000000000000000000':
        in_eth_raw_amount = int(tx['input'][650:714], 16)
        in_eth_decimals = 18
        eth_amount = in_eth_raw_amount / (10 ** in_eth_decimals)
        
    else:
        out_eth_raw_amount = int(tx['input'][-316:-252], 16)
        out_eth_decimals = 18
        eth_amount = (out_eth_raw_amount / (10 ** out_eth_decimals)) * 1.008
        
        #in_token_address = '0x' + tx['input'][546:586]
        #in_token_raw_amount = Web3.to_int['input'][650:714]
        #in_token_decimals = ...
        #in_token_amount = in_token_raw_amount/ 10 ** in_token_decimals

    return [tx['transaction_hash'], eth_amount]

In [377]:
Web3.to_int(0x58857af8583b865025e14bd31)
#len('00000000000000000000000000000000000000000000000000000007b80b2a13e8645a')

438336779330374001319298252081

In [366]:
str1 = '0x5f57552900000000000000000000000000000000000000000000000000000000000000800000000000000000000000006982508145454ce325ddbe47a25d4ec3d23119330000000000000000000000000000000000000000026c62ad77dc602dae00000000000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000136f6e65496e6368563546656544796e616d69630000000000000000000000000000000000000000000000000000000000000000000000000000000000000002000000000000000000000000006982508145454ce325ddbe47a25d4ec3d231193300000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000026c62ad77dc602dae00000000000000000000000000000000000000000000000000000007a6c0d3a12c704600000000000000000000000000000000000000000000000000000000000001200000000000000000000000000000000000000000000000000011a4abcde45c53000000000000000000000000f326e4de8f66a0bdc0970b79e0924e33c79f1915000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000c80502b1c50000000000000000000000006982508145454ce325ddbe47a25d4ec3d23119330000000000000000000000000000000000000000026c62ad77dc602dae00000000000000000000000000000000000000000000000000000007b80b2a13e8645a0000000000000000000000000000000000000000000000000000000000000080000000000000000000000000000000000000000000000000000000000000000140000000000000003b6d0340a43fe16908251ee70ef74718545e4fe6c5ccec9fab4991fe00000000000000000000000000000000000000000000000000c7'
str1[-316:-252]
int('00000000000000000000000000000000000000000000000007b80b2a13e8645a', 16)

556206829330785370

In [421]:
volume = {'unoswap':0, 'unoswap_count':0 ,'univ3':0, 'swap':0}
counter_max = 10
counter = 0

#df_transaction = pd.DataFrame(columns=['tx_hash', 'in_token_address', 'in_token_amount', 'out_token_address', 'out_token_amount'])


db = pd.DataFrame(columns=['tx_hash', 'volume'])

for tx in one_inch_tx:
    #if counter >= counter_max:
        #break
    
    if oneinch_unoswap in tx['input']:
        tx_volume = one_inch_unoswap_resolve(tx)
        
        tmp = {'tx_hash': tx_volume[0], 'volume': tx_volume[1]}
        db.loc[len(db)]=tmp
        
        volume['unoswap'] += tx_volume[1]
        volume['unoswap_count'] += 1
    #elif oneinch_univ3 in tx:
        #continue
    #elif oneinch_swap in tx:
        #continue
    #else:
        #continue

In [422]:
volume

{'unoswap': 22302631065017.816, 'unoswap_count': 790, 'univ3': 0, 'swap': 0}

In [388]:
for tx in one_inch_tx:
    if tx['transaction_hash'] == '0x409cb7ec387f3b06e4b43f89b777a163a00d31a8283e089cfaffa15b39b30269':
        a = tx

In [431]:
one_inch_tx[1]['input'][-316:-252]

'000000000000000000000000000000000000000000000010566466a3b39cf782'

In [433]:
Web3.to_int(0x10566466a3b39cf782)

301373118617564804994

In [430]:
one_inch_unoswap_resolve(one_inch_tx[1])

['0x5684b195fa5e863a1b55ffc723635bf92c748db8cb360b26cb524319ddf3c413',
 301.3731186175648]

In [328]:
#oneinch_unoswap in tx
#oneinch_unoswap in one_inch_tx[0]['input']
volume['unoswap']

22481035247828.65

In [329]:
volume

{'unoswap': 22481035247828.65, 'univ3': 0, 'swap': 0}

In [226]:
one_inch_tx[3]['input'][546:586] # Out_Token_Address

'ea7b7dc089c9a4a916b5a7a37617f59fd54e37e4'

In [284]:
#one_inch_tx[1]['input'][546+64:586+64] # In Token Amount
one_inch_tx[1]

{'block_hash': '0xa008e6ab9b1c675a74342d8731d352702d6138868a8ce77fbaa671241e4232c8',
 'block_number': '17363590',
 'block_timestamp': '2023-05-29 08:54:47',
 'contract_address': '',
 'cumulative_gas_used': '8626568',
 'effective_gas_price': '25187064738',
 'from_address': '0x5ab7f9fdfa7cdadddd991cf535f78608fe767549',
 'gas': '231650',
 'gas_price': '25187064738',
 'gas_used': '187060',
 'input': '0x5f5755290000000000000000000000000000000000000000000000000000000000000080000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000e6ed27d666800000000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000136f6e65496e6368563546656544796e616d696300000000000000000000000000000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000000000000000000000000000008ce9137d39326ad0cd6491fb5cc0cba0e089b6a900000000000000